# 📘 Архитектуры Агентов 13: Parallel Exploration + Ensemble Decision (Параллельное Исследование + Ансамблевое Решение)

Добро пожаловать в глубокое погружение в одну из самых надежных и устойчивых архитектур рассуждения: **Parallel Exploration with Ensemble Decision-Making** (Параллельное исследование с принятием ансамблевых решений). Этот паттерн решает проблему присущего недетерминизма и потенциальных предвзятостей (biases) одной LLM, используя принцип "мудрости толпы", примененный к ИИ-агентам.

Вместо того чтобы полагаться на единственную линию рассуждений, эта архитектура порождает несколько независимых агентов для анализа проблемы с разных точек зрения одновременно. Каждый агент следует своему собственному пути рассуждений, подобно разным экспертам в комитете. Их индивидуальные выводы затем собираются и синтезируются финальным агентом "агрегатором", который взвешивает различные точки зрения, определяет консенсус и конфликты, и производит окончательный, более нюансированный и надежный ответ.

Чтобы построить сложную и мощную реализацию, мы создадим **имитацию Инвестиционного Комитета ИИ**, которому поручено ответить на сложный, открытый вопрос: **"Является ли NVIDIA (NVDA) хорошей долгосрочной инвестицией в середине 2024 года?"**

Наш комитет будет состоять из трех различных параллельных агентов:
1.  **The Bullish Growth Analyst (Аналитик бычьего роста):** Оптимист, который фокусируется на инновациях, доминировании на рынке и будущем потенциале.
2.  **The Cautious Value Analyst (Осторожный стоимостный аналитик):** Скептик, который тщательно изучает финансовые показатели, оценку, конкуренцию и потенциальные риски.
3.  **The Quantitative Analyst (Quant / Квант):** Эксперт, ориентированный на данные, который смотрит исключительно на финансовые метрики и технические индикаторы акций.

Наконец, агент **Chief Investment Officer (CIO / Директор по инвестициям)** синтезирует их противоречивые отчеты в окончательный, сбалансированный инвестиционный тезис, предоставляя гораздо более надежный ответ, чем любой отдельный агент мог бы дать в одиночку.

### Определение
**Parallel Exploration + Ensemble Decision** — это агентная архитектура, где проблема одновременно обрабатывается несколькими независимыми агентами или путями рассуждения. Индивидуальные результаты затем агрегируются, часто отдельным агентом, с помощью метода вроде голосования, достижения консенсуса или синтеза, чтобы прийти к финальному, более надежному заключению.

### Высокоуровневый рабочий процесс

1.  **Fan-Out (Параллельное исследование):** Запрос пользователя распределяется между N независимыми агентами-специалистами. Критически важно, что этим агентам часто даются различные инструкции, персоны или инструменты, чтобы поощрять разнообразные аналитические подходы.
2.  **Independent Processing (Независимая обработка):** Каждый агент работает над проблемой изолированно, генерируя свой собственный полный анализ, заключение или ответ.
3.  **Fan-In (Агрегация):** Результаты от всех N агентов собираются.
4.  **Synthesize (Ансамблевое решение):** Финальный агент "агрегатор" или "судья" получает все индивидуальные результаты. Его задача — проанализировать эти точки зрения, определить общие черты, взвесить противоречивые доказательства и синтезировать всеобъемлющий финальный ответ.

### Когда использовать / Применения
*   **Сложные вопросы и ответы (Q&A) требующие рассуждения:** Для сложных, неоднозначных вопросов, где одна линия рассуждений может легко упустить нюансы (например, "Какова была основная причина финансового кризиса 2008 года?").
*   **Проверка фактов и верификация:** Наличие нескольких агентов, ищущих и проверяющих факт из разных источников, может радикально снизить галлюцинации.
*   **Поддержка принятия решений с высокими ставками:** В таких областях, как медицина или финансы, получение "второго мнения" (или третьего, или четвертого) от разных персон ИИ перед вынесением рекомендации.

### Сильные и слабые стороны
*   **Сильные стороны:**
    *   **Повышает надежность и точность:** Усредняет случайные ошибки или предвзятости одного агента, делая финальный ответ гораздо более вероятным быть правильным и всесторонним.
    *   **Снижает галлюцинации:** Если один агент галлюцинирует факт, другие вряд ли сделают то же самое, и агрегатор может легко обнаружить "выброс".
*   **Слабые стороны:**
    *   **Очень высокая стоимость:** Это одна из самых дорогих архитектур, так как она умножает количество вызовов LLM на количество агентов в ансамбле (плюс финальный вызов агрегации).
    *   **Увеличенная задержка (Latency):** Система должна ждать, пока завершатся все параллельные пути, прежде чем начнется финальный синтез.

## Фаза 0: Основа и Настройка

Мы установим библиотеки и настроим нашу среду. Нам понадобится `langchain-tavily` для исследовательских инструментов наших аналитиков.

In [ ]:
import os
import json
from typing import List, Annotated, TypedDict, Optional, Dict, Any, Union
from dotenv import load_dotenv

# Pydantic для структурированных выводов
from pydantic import BaseModel, Field

# Компоненты LangChain/Nebius
from langchain_nebius import ChatNebius
from langchain_community.tools.tavily_search import TavilySearchResults as TavilySearch
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown

from notebook_utils import setup_environment

# --- Настройка API ключей и трассировки ---
console = setup_environment(
    project_name="Agentic Architecture - Parallel Ensemble (Nebius)",
    required_keys=["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"]
)

In [ ]:
import os
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Pydantic для моделирования данных
from pydantic import BaseModel, Field

# Компоненты LangChain
from langchain_nebius import ChatNebius
from langchain_tavily import TavilySearch
from langchain_core.prompts import ChatPromptTemplate

# Компоненты LangGraph
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel

# --- Настройка API ключей и трассировки ---
load_dotenv(dotenv_path="../.env")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Parallel Ensemble (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Переменные окружения загружены и трассировка настроена.")

Environment variables loaded and tracing is set up.


## Фаза 1: Создание Разнообразных Аналитиков-Специалистов

Ключ к успешному ансамблю — когнитивное разнообразие. Мы создадим трех разных агентов-аналитиков, каждого с детальной персоной (persona), разработанной для создания разного вида анализа. Все будут иметь доступ к инструменту поиска.

In [ ]:
console = Console()
# Для этой сложной задачи нужна мощная модель
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0.3)
search_tool = TavilySearch(max_results=5)

# Состояние LangGraph
class EnsembleState(TypedDict):
    query: str
    # Словарь analyses будет хранить вывод от каждого параллельного агента
    analyses: Dict[str, str]
    final_recommendation: Optional[Any] # Будет хранить структурированный вывод от CIO

# Помощник-фабрика для создания узлов наших аналитиков
def create_analyst_node(persona: str, agent_name: str):
    """Фабрика для создания узла специалиста-аналитика с уникальной персоной."""
    system_prompt = f"You are an expert financial analyst. Your persona is '{persona}'. You must use your search tool to gather up-to-date information. Based on your persona and research, provide a detailed investment analysis for the user's query. Conclude with a clear 'Recommendation' (e.g., Buy, Hold, Sell) and a 'Confidence Score' (1-10)."
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{query}")
    ])
    chain = prompt | llm.bind_tools([search_tool])
    
    def analyst_node(state: EnsembleState) -> Dict[str, Any]:
        console.print(f"--- 👨‍💻 Calling {agent_name} --- ")
        result = chain.invoke({"query": state['query']})
        # Обновление состояния тщательно спроектировано, чтобы добавлять в словарь 'analyses'
        # без перезаписи других. Это ключевой момент для параллельного выполнения.
        current_analyses = state.get('analyses', {})
        current_analyses[agent_name] = result.content
        return {"analyses": current_analyses}
    
    return analyst_node

# 1. The Bullish Growth Analyst (Аналитик бычьего роста)
bullish_persona = "The Bullish Growth Analyst: You are extremely optimistic about technology and innovation. You focus on Total Addressable Market (TAM), visionary leadership, technological moats, and future growth potential. Downplay short-term volatility and valuation concerns in favor of the long-term disruptive story."
bullish_analyst_node = create_analyst_node(bullish_persona, "BullishAnalyst")

# 2. The Cautious Value Analyst (Осторожный стоимостный аналитик)
value_persona = "The Cautious Value Analyst: You are a skeptical investor focused on fundamentals and risk. You scrutinize financial statements, P/E ratios, debt levels, and competitive threats. You are wary of hype and market bubbles. Highlight potential risks, downside scenarios, and reasons for caution."
value_analyst_node = create_analyst_node(value_persona, "ValueAnalyst")

# 3. The Quantitative Analyst (Квант)
quant_persona = "The Quantitative Analyst (Quant): You are purely data-driven. You ignore narratives and focus on hard numbers. Report on key financial metrics (YoY revenue growth, EPS, margins), valuation multiples (P/E, P/S), and technical indicators (RSI, moving averages). Your analysis must be objective and based on the data you find."
quant_analyst_node = create_analyst_node(quant_persona, "QuantAnalyst")

print("Агенты-аналитики специалисты успешно определены.")

Specialist analyst agents defined successfully.


## Фаза 2: Создание Агента-Агрегатора CIO

Это шаг **Ensemble Decision** (Ансамблевое решение). Мы создадим финального агента, Директора по инвестициям (CIO), чья работа — синтезировать отчеты от трех аналитиков. Этому агенту нужен сложный промпт и модель структурированного вывода, чтобы гарантировать, что он выдаст высококачественную, сбалансированную финальную рекомендацию.

In [ ]:
# Pydantic модель для финальной, структурированной рекомендации
class FinalRecommendation(BaseModel):
    """Финальный, синтезированный инвестиционный тезис от CIO."""
    final_recommendation: str = Field(description="Финальное инвестиционное решение, должно быть одним из 'Strong Buy', 'Buy', 'Hold', 'Sell', 'Strong Sell'.")
    confidence_score: float = Field(description="Уверенность CIO в этой рекомендации, от 1.0 до 10.0.")
    synthesis_summary: str = Field(description="Подробное резюме, синтезирующее точки зрения аналитиков, выделяющее пункты согласия и споров.")
    identified_opportunities: List[str] = Field(description="Маркированный список основных возможностей или бычьих аргументов.")
    identified_risks: List[str] = Field(description="Маркированный список основных рисков или медвежьих аргументов.")

def cio_synthesizer_node(state: EnsembleState) -> Dict[str, Any]:
    """Финальный узел, который синтезирует все анализы в единую рекомендацию."""
    console.print("--- 🏛️ Вызов Директора по инвестициям (CIO) для Финального Решения ---")
    
    # Объединяем все индивидуальные анализы в одну строку для промпта
    all_analyses = "\n\n---\n\n".join(
        f"**Analysis from {name}:**\n{analysis}"
        for name, analysis in state['analyses'].items()
    )
    
    cio_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are the Chief Investment Officer (CIO) of a major investment fund. You have received reports from your team of specialist analysts. Your task is to synthesize these diverse and often conflicting viewpoints into a single, final, and actionable investment thesis. You must weigh the growth potential against the risks and valuation concerns to arrive at a balanced, well-reasoned conclusion."),
        ("human", "Here are the reports from your team regarding the query: '{query}'\n\n{analyses}\n\nBased on all these perspectives, provide your final, synthesized investment thesis.")
    ])
    
    cio_llm = llm.with_structured_output(FinalRecommendation)
    chain = cio_prompt | cio_llm
    
    final_decision = chain.invoke({"query": state['query'], "analyses": all_analyses})
    
    return {"final_recommendation": final_decision}

print("Агент-агрегатор CIO успешно определен.")

CIO Aggregator agent defined successfully.


## Фаза 3: Сборка Рабочего Процесса LangGraph

Теперь мы связываем все вместе. Граф будет иметь единую точку входа, которая разветвляется (fans out) к трем нашим параллельным узлам аналитиков. Как только все аналитики завершат свою работу, граф снова сойдется (fan back in) к единственному узлу синтезатора CIO, который произведет финальный результат.

In [ ]:
# Входной узел просто берет запрос и подготавливает состояние.
def start_analysis_node(state: EnsembleState) -> Dict[str, Any]:
    # Инициализация словаря анализов
    return {"analyses": {}}

# Построение графа
workflow = StateGraph(EnsembleState)

workflow.add_node("start_analysis", start_analysis_node)

# Добавление параллельных узлов аналитиков
workflow.add_node("bullish_analyst", bullish_analyst_node)
workflow.add_node("value_analyst", value_analyst_node)
workflow.add_node("quant_analyst", quant_analyst_node)

# Добавление финального узла синтезатора
workflow.add_node("cio_synthesizer", cio_synthesizer_node)

# Установка точки входа
workflow.set_entry_point("start_analysis")

# FAN-OUT (Разветвление): Со старта запускаем всех трех аналитиков параллельно
workflow.add_edge("start_analysis", ["bullish_analyst", "value_analyst", "quant_analyst"])

# FAN-IN (Схождение): После того, как все аналитики закончат, вызываем синтезатора CIO
workflow.add_edge(["bullish_analyst", "value_analyst", "quant_analyst"], "cio_synthesizer")

workflow.add_edge("cio_synthesizer", END)

ensemble_agent = workflow.compile()
print("Граф агента Параллельного Ансамбля успешно скомпилирован.")

Parallel Ensemble agent graph compiled successfully.


## Фаза 4: Демонстрация и Анализ

Давайте запустим полный инвестиционный комитет по нашему сложному вопросу. Сначала мы выведем индивидуальные отчеты, чтобы увидеть разнообразие мнений, а затем финальную синтезированную рекомендацию CIO.

In [ ]:
query = "Based on recent news, financial performance, and future outlook, is NVIDIA (NVDA) a good long-term investment in mid-2024?"
console.print(f"--- 📈 Запуск Инвестиционного Комитета для: {query} ---")

result = ensemble_agent.invoke({"query": query})

# Отображение индивидуальных отчетов
console.print("\n--- Индивидуальные отчеты аналитиков ---")
for name, analysis in result['analyses'].items():
    console.print(Panel(Markdown(analysis), title=f"[bold yellow]{name}[/bold yellow]", border_style="yellow"))

# Отображение финальной синтезированной рекомендации
console.print("\n--- Финальная рекомендация CIO ---")
final_rec = result['final_recommendation']
if final_rec:
    rec_panel = Panel(
        f"[bold]Final Recommendation:[/bold] {final_rec.final_recommendation}\n"
        f"[bold]Confidence Score:[/bold] {final_rec.confidence_score}/10\n\n"
        f"[bold]Synthesis Summary:[/bold]\n{final_rec.synthesis_summary}\n\n"
        f"[bold]Identified Opportunities:[/bold]\n* {'\n* '.join(final_rec.identified_opportunities)}\n\n"
        f"[bold]Identified Risks:[/bold]\n* {'\n* '.join(final_rec.identified_risks)}",
        title="[bold green]Тезис Директора по инвестициям (CIO)[/bold green]",
        border_style="green"
    )
    console.print(rec_panel)

--- 📈 Running Investment Committee for: Is NVIDIA (NVDA) a good long-term investment in mid-2024? ---


--- 👨‍💻 Calling BullishAnalyst --- 
--- 👨‍💻 Calling ValueAnalyst --- 
--- 👨‍💻 Calling QuantAnalyst --- 
--- 🏛️ Calling Chief Investment Officer for Final Decision ---



--- Individual Analyst Reports ---


Analysis from BullishAnalyst:
NVIDIA's position as the undisputed leader in accelerated computing for AI makes it an incredibly compelling long-term investment. The recent announcements of their next-generation Rubin architecture, hot on the heels of the Blackwell platform, demonstrates an unprecedented pace of innovation that competitors simply cannot match. Their CUDA software ecosystem creates a deep and durable moat, locking in developers and enterprises. The Total Addressable Market for AI is projected to be in the trillions, and NVIDIA is poised to capture a significant portion of this. While short-term volatility is always a factor, the visionary leadership of Jensen Huang and the company's clear roadmap for creating a new era of 'AI factories' points to a future of sustained, exponential growth. Any concerns about valuation are secondary to the sheer scale of the technological revolution they are leading.

Recommendation: Buy
Confidence Score: 9/10

Analysis from ValueAnalyst:
While NVIDIA's technological prowess is undeniable, a cautious approach is warranted due to its astronomical valuation. The stock is trading at extremely high multiples (P/E and P/S ratios) that price in not just perfection, but a future of flawless, uninterrupted hyper-growth. This leaves very little margin for error. Several key risks must be considered: 1) Increased competition from other major tech players (AMD, Intel) and in-house chip designs from hyperscalers (Google, Amazon). 2) Geopolitical risks, particularly concerning supply chains and regulations around sales to China. 3) The cyclical nature of the semiconductor industry, which could see a downturn if the current AI spending boom slows. While the company is a market leader, the current stock price appears to have priced in years of future growth, making it vulnerable to a significant correction if any of these risks materialize.

Recommendation: Hold
Confidence Score: 7/10

Analysis from QuantAnalyst:
Based on current data, NVIDIA exhibits the following quantitative profile:
- **Financials:** Revenue growth YoY for the most recent quarter exceeded 260%. Earnings Per Share (EPS) have shown similar explosive growth. Gross margins are exceptionally high for a hardware company, currently in the high 70s percentile.
- **Valuation:** The forward Price-to-Earnings (P/E) ratio is approximately 45-50x, which is high relative to the broader market but may be justifiable given the growth rate (PEG ratio is closer to 1.5). The Price-to-Sales (P/S) ratio is also elevated, above 30x.
- **Technicals:** The stock is currently trading well above its 50-day and 200-day moving averages, indicating a strong bullish trend. However, the Relative Strength Index (RSI) is frequently in the overbought territory (>70), suggesting the stock may be due for a short-term pullback or consolidation.

Recommendation: Hold
Confidence Score: 8/10


--- Final CIO Recommendation ---


Final Recommendation: Buy
Confidence Score: 7.5
Synthesis Summary: The committee presents a compelling but contested case for NVIDIA. There is unanimous agreement on the company's current technological dominance and extraordinary financial performance, as highlighted by both the Bullish and Quant analysts. However, the Value and Quant analysts raise critical, concurring points about the stock's extremely high valuation and the potential for volatility, as indicated by its overbought RSI. The Bullish case hinges on the belief that the AI revolution is a paradigm shift that justifies these multiples, while the Cautious case argues that the current price leaves no room for execution error or unforeseen macroeconomic headwinds. The consensus is that NVIDIA is a phenomenal company, but the stock is a risky proposition at its current price. Therefore, the final recommendation is a 'Buy', but with a strong emphasis on it being a long-term position and advising a cautious entry, perhaps by dol

### Анализ результатов

Демонстрация мощно иллюстрирует ценность этой сложной архитектуры:

1.  **Когнитивное разнообразие:** Три аналитика выдали совершенно разные, но индивидуально валидные отчеты. "Бык" сосредоточился на грандиозном видении, аналитик "Стоимости" — на рисках, а "Квант" предоставил жесткие данные. Один агент, даже с нейтральным промптом, вероятно, склонился бы в одну из этих сторон, давая неполную картину.

2.  **Надежный синтез:** Агент CIO не просто "усредил" рекомендации ('Покупать', 'Держать', 'Держать'). Вместо этого он выполнил настоящий синтез. Он признал обоснованность бычьего сценария, но сбалансировал его опасениями аналитиков стоимости и количественного анализа по поводу оценки. Финальная рекомендация с определенным уровнем уверенности отражает этот нюанс, эффективно говоря: "Это отличная компания, но акции дороги, поэтому действуйте с осторожностью".

3.  **Практичные и объяснимые инсайты:** Финальный структурированный вывод с четкими списками возможностей и рисков гораздо полезнее для человека, принимающего решения, чем единый монолитный блок текста. Он объясняет, *почему* была дана финальная рекомендация, показывая, как были взвешены различные экспертные мнения.

Этот метод ансамбля успешно трансформировал субъективный и сложный вопрос в хорошо обоснованный, многоаспектный анализ, значительно повышая надежность и доверие к конечному результату по сравнению с любым одиночным агентом.

## Заключение

В этом блокноте мы реализовали всестороннего и сложного агента **Parallel Exploration + Ensemble Decision**. Симулируя комитет из разнообразных экспертов и финального лица, принимающего решения, мы построили систему, которая превосходит других в решении неоднозначных задач с высокими ставками.

Основные принципы — **порождение разнообразных, независимых рассуждающих агентов** и затем **синтез их результатов** — создают мощный механизм для смягчения предвзятости, уменьшения ошибок и увеличения глубины анализа. Хотя это одна из самых вычислительно дорогих агентных архитектур, её способность выдавать надежные, устойчивые и нюансированные выводы делает её незаменимым инструментом для любого приложения, где качество и достоверность финального решения имеют первостепенное значение.